<a href="https://colab.research.google.com/github/wasimafser/Amazon-Review-Scrapper/blob/main/amazon_review_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
 
GIVEN_URL = "https://www.amazon.co.uk/All-New-Fire-Tablet-Alexa-Display/dp/B07952CV7L?pd_rd_w=p2svY&pf_rd_p=6d00cb1b-e522-41f6-98b2-97052078725d&pf_rd_r=6c41a23c-1c43-410a-8646-a14bdcfa1561&pd_rd_r=6c41a23c-1c43-410a-8646-a14bdcfa1561&pd_rd_wg=w9uHe&ref_=pd_gw_unk"
HEADERS = ({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
REVIEWS_URL = None
 
PAGE = 1
 
def get_content(url=None):
  global REVIEWS_URL
  if not url:
    url = GIVEN_URL
  response = requests.get(url, headers=HEADERS)
  content = BeautifulSoup(response.content, "lxml")
  if url == GIVEN_URL:
    REVIEWS_URL = f"https://www.amazon.co.uk{content.find('a', {'data-hook': 'see-all-reviews-link-foot'}).get('href')}"
    get_content(url=REVIEWS_URL+"&pageNumber=1")
  return content
 
with open("output.csv", "w") as outfile:
  writer = csv.DictWriter(outfile, fieldnames=["Review Title","Review Description","No Of Stars","Review Given By","Date of Review"], quoting=csv.QUOTE_ALL)
  writer.writeheader()
  content = get_content()
  reviews = content.findAll("div", {'data-hook': 'review'})
  while len(reviews) > 0:
    for review in reviews:
      template = {}
      template['Review Title'] = review.find('a', {'data-hook': 'review-title'}).span.text
      template['Review Description'] = review.find('span', {'data-hook': 'review-body'}).span.text
      template['No Of Stars'] = review.find('i', {'data-hook': 'review-star-rating'}).span.text.split(" ")[0]
      template['Review Given By'] = review.find('span', {'class': 'a-profile-name'}).text
      template['Date of Review'] = review.find('span', {'data-hook': 'review-date'}).text.split("on")[-1]
      writer.writerow(template)
    PAGE += 1
    content = get_content(url=REVIEWS_URL+f"&pageNumber={PAGE}")
    reviews = content.findAll("div", {'data-hook': 'review'})